## Cartpole DQN
Cartpole agent using Deep Q-Learning Network with Keras and Open AI Gym

#### Select processing devices

In [2]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

#### Import dependencies

In [3]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # to create directories

Using TensorFlow backend.
C:\Users\dineth.r\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dineth.r\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dineth.r\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

### Set parameters

In [4]:
env = gym.make('CartPole-v0') # initialise environment

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001 # n games we want agent to play (default - 1001)

print(state_size,",",action_size)

4 , 2


In [5]:
output_dir = 'models/cartpole/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Define Agent

In [6]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in additions to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost
        self.model = self._build_model() # private method
    
    # Neural net to approximate Q-value function
    def _build_model(self):
        
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions -> 2 output neurons -> 0/1 (L/R)
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later
        
    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0])
    
    # method that trains NN with experiences sampled from memory
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
                # (target) = reward + (discount rate gamma) * (maximum target Q based on future action a')
                
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)
        
    def save(self, name):
        self.model.save_weights(name)

In [7]:
# Initialize agent
agent = DQNAgent(state_size, action_size)

Instructions for updating:
Colocations handled automatically by placer.


### Interact with environment

In [8]:
done = False # by default state game episode has not ended
for e in range(n_episodes):
    state = env.reset() # reset state at start of each new episode of the game
    state = np.reshape(state, [1, state_size])
    
    for time in range(5000): # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
#         env.render()
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position
        reward = reward if not done else -10 # reward +1 for each additional frame with pole upright
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.
        
        state = next_state # set "current state" for upcoming iteration to the current next state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, agent.epsilon))
            break
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")

episode: 0/1001, score: 21, e: 1.0
episode: 1/1001, score: 37, e: 1.0
Instructions for updating:
Use tf.cast instead.
episode: 2/1001, score: 34, e: 0.99
episode: 3/1001, score: 14, e: 0.99
episode: 4/1001, score: 10, e: 0.99
episode: 5/1001, score: 16, e: 0.98
episode: 6/1001, score: 68, e: 0.98
episode: 7/1001, score: 44, e: 0.97
episode: 8/1001, score: 16, e: 0.97
episode: 9/1001, score: 16, e: 0.96
episode: 10/1001, score: 24, e: 0.96
episode: 11/1001, score: 19, e: 0.95
episode: 12/1001, score: 19, e: 0.95
episode: 13/1001, score: 12, e: 0.94
episode: 14/1001, score: 10, e: 0.94
episode: 15/1001, score: 39, e: 0.93
episode: 16/1001, score: 18, e: 0.93
episode: 17/1001, score: 36, e: 0.92
episode: 18/1001, score: 34, e: 0.92
episode: 19/1001, score: 51, e: 0.91
episode: 20/1001, score: 29, e: 0.91
episode: 21/1001, score: 11, e: 0.9
episode: 22/1001, score: 14, e: 0.9
episode: 23/1001, score: 21, e: 0.9
episode: 24/1001, score: 36, e: 0.89
episode: 25/1001, score: 12, e: 0.89
episo

episode: 213/1001, score: 55, e: 0.35
episode: 214/1001, score: 48, e: 0.34
episode: 215/1001, score: 46, e: 0.34
episode: 216/1001, score: 49, e: 0.34
episode: 217/1001, score: 38, e: 0.34
episode: 218/1001, score: 64, e: 0.34
episode: 219/1001, score: 20, e: 0.34
episode: 220/1001, score: 42, e: 0.33
episode: 221/1001, score: 26, e: 0.33
episode: 222/1001, score: 30, e: 0.33
episode: 223/1001, score: 34, e: 0.33
episode: 224/1001, score: 40, e: 0.33
episode: 225/1001, score: 27, e: 0.33
episode: 226/1001, score: 69, e: 0.32
episode: 227/1001, score: 31, e: 0.32
episode: 228/1001, score: 93, e: 0.32
episode: 229/1001, score: 37, e: 0.32
episode: 230/1001, score: 68, e: 0.32
episode: 231/1001, score: 41, e: 0.32
episode: 232/1001, score: 38, e: 0.31
episode: 233/1001, score: 33, e: 0.31
episode: 234/1001, score: 17, e: 0.31
episode: 235/1001, score: 24, e: 0.31
episode: 236/1001, score: 20, e: 0.31
episode: 237/1001, score: 25, e: 0.31
episode: 238/1001, score: 22, e: 0.3
episode: 239/

episode: 632/1001, score: 174, e: 0.042
episode: 633/1001, score: 151, e: 0.042
episode: 634/1001, score: 140, e: 0.042
episode: 635/1001, score: 159, e: 0.042
episode: 636/1001, score: 157, e: 0.041
episode: 637/1001, score: 125, e: 0.041
episode: 638/1001, score: 157, e: 0.041
episode: 639/1001, score: 178, e: 0.041
episode: 640/1001, score: 146, e: 0.041
episode: 641/1001, score: 199, e: 0.04
episode: 642/1001, score: 75, e: 0.04
episode: 643/1001, score: 199, e: 0.04
episode: 644/1001, score: 147, e: 0.04
episode: 645/1001, score: 157, e: 0.04
episode: 646/1001, score: 167, e: 0.039
episode: 647/1001, score: 155, e: 0.039
episode: 648/1001, score: 192, e: 0.039
episode: 649/1001, score: 174, e: 0.039
episode: 650/1001, score: 199, e: 0.039
episode: 651/1001, score: 199, e: 0.038
episode: 652/1001, score: 92, e: 0.038
episode: 653/1001, score: 175, e: 0.038
episode: 654/1001, score: 131, e: 0.038
episode: 655/1001, score: 116, e: 0.038
episode: 656/1001, score: 195, e: 0.038
episode

episode: 840/1001, score: 108, e: 0.015
episode: 841/1001, score: 160, e: 0.015
episode: 842/1001, score: 199, e: 0.015
episode: 843/1001, score: 199, e: 0.015
episode: 844/1001, score: 178, e: 0.015
episode: 845/1001, score: 175, e: 0.015
episode: 846/1001, score: 158, e: 0.014
episode: 847/1001, score: 170, e: 0.014
episode: 848/1001, score: 177, e: 0.014
episode: 849/1001, score: 159, e: 0.014
episode: 850/1001, score: 106, e: 0.014
episode: 851/1001, score: 199, e: 0.014
episode: 852/1001, score: 133, e: 0.014
episode: 853/1001, score: 142, e: 0.014
episode: 854/1001, score: 160, e: 0.014
episode: 855/1001, score: 170, e: 0.014
episode: 856/1001, score: 154, e: 0.014
episode: 857/1001, score: 150, e: 0.014
episode: 858/1001, score: 161, e: 0.014
episode: 859/1001, score: 66, e: 0.014
episode: 860/1001, score: 122, e: 0.013
episode: 861/1001, score: 18, e: 0.013
episode: 862/1001, score: 61, e: 0.013
episode: 863/1001, score: 126, e: 0.013
episode: 864/1001, score: 171, e: 0.013
epi